In [ ]:
!rm *.fbs *.npy *.json *.bin

In [ ]:
# Generate 2 structure flatbuffers
!./converters/StructureGenerator.py -f ./res/posStruc.npy --num_structs 1 --num_orientations 1
!flatc -t converters/schemas/Structure.fbs -- struct_0.fbs --raw-binary --strict-json

In [ ]:
# generate a parameter config (of interest is defocus [0.5,4,N] where N is number of steps)
# output file to demo_params.fbs
!./converters/ParameterGenerator.py -a [0.1] \
 -d [0.5,4,25] \
 -bf [1.0] \
 -id [7.0,8.0,9] \
 -np [128] \
 -pw [0.3] \
 -s [1.0] \
 -ew [0.019866] \
 -snr [1.0] \
 -ep [25.0,26.0,27] \
 -seed [28.0,29.0,30] \
 -st [31.0,32.0,33] \
 -c [34.0,35.0,36] \
 -o demo_params.fbs

!flatc -t converters/schemas/ParamModel.fbs -- demo_params.fbs --raw-binary --strict-json

In [ ]:
# generate images from struct_0 and output to images_0
!./DataGenerator.py images -p demo_params.fbs -s struct_0.fbs -o images_0.fbs
!flatc -t converters/schemas/DataSet.fbs -- images_0.fbs --raw-binary --strict-json
# generate images from struct_1 and output to images_1
# !./DataGenerator.py images -p demo_params.fbs -s struct_1.fbs -o images_1.fbs

In [ ]:
# generate ctfs using the demo parameters
!./DataGenerator.py ctfs -p demo_params.fbs

In [ ]:
# synthesize images_0 using ctfs and output all generated permutations with leading tag STRUCT0
!./DataGenerator.py synth -i images_0.fbs -c ctfs.fbs -o STRUCT0

# from res.inspector import *

In [ ]:
# convert any dataset flatbuffer to numpy array of complex values
from res.inspector import *
# filename structure := TAG_synth_dataset_ID.fbs
serialize_fbs_dataset("STRUCT0_synth_dataset_0.fbs")
serialize_fbs_dataset("images_0.fbs")

In [ ]:
# peek at the files
f =["serialized_STRUCT0_synth_dataset_0.npy", "serialized_images_0.npy"]
ID = 1
import matplotlib.pyplot as plt
serialized_synth_imgs = np.load(f[ID])
serialized_imgs = np.load(f[ID])
r,c = 5,5
fig, axes = plt.subplots(r, c)
for i in range(r):
    for j in range(c):
      axes[i,j].imshow(np.real(serialized_synth_imgs[i * r + j]))
axes[r-1,c-1].imshow(np.real(serialized_imgs[0]), cmap="gray")

plt.show()

In [ ]:
ctf_paramSet_buf = ParametersT.InitFromObj(
    DataSet.GetRootAsDataSet(dl.load_flatbuffer("ctfs.fbs"), 0).Params()
)
serialized_ctfs = np.load("serialized_STRUCT0_synth_dataset_0.npy")

row = 5
col = 5
f = 0
fig, axes = plt.subplots(row, col)
for i in range(row):
    for j in range(col):
        # print())
        axes[i, j].imshow(np.real(np.fft.fftshift(serialized_ctfs[(i + f) * row + j])))
        # axes[i,j].imshow(get_phase(np.fft.fftshift(b[(i+f)*row+j]),128),cmap="Greys")
        axes[i, j].set_title(f"{ctf_paramSet_buf.defocus[(i+f)*row+j]}", fontsize=6)